In [2]:
! pip install cssselect

In [3]:
import requests
import numpy as np
import pandas as pd
import json
import joblib
import cssselect
import lxml.html
from tqdm import tqdm

In [20]:
train_small = pd.read_csv('/Users/baeyuna/Documents/SNU_Dlab/Data/MINDsmall_train/news.tsv', delimiter='\t', header=None)
print(train_small.shape)
train_small.head()

(51282, 8)


,0,1,2,3,4,5,6,7
0,N48100,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://www.msn.com/en-us/lifestyle/lifestyler...,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N54564,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://www.msn.com/en-us/health/weightloss/50...,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N2942,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N14733,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://www.msn.com/en-us/health/voices/i-was-...,[],"[{""Label"": ""National Basketball Association"", ..."
4,N31455,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://www.msn.com/en-us/health/medical/how-t...,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


# 데이터 파싱을 위한 html 링크 생성

In [22]:
train_small_urls = train_small.iloc[:,5].tolist()
print(len(train_small_urls))
train_small_urls[:3]

51282


['https://www.msn.com/en-us/lifestyle/lifestyleroyals/the-brands-queen-elizabeth,-prince-charles,-and-prince-philip-swear-by/ss-AAGH0ET?ocid=chopendata',
 'https://www.msn.com/en-us/health/weightloss/50-worst-habits-for-belly-fat/ss-AAB19MK?ocid=chopendata',
 'https://www.msn.com/en-us/news/world/the-cost-of-trumps-aid-freeze-in-the-trenches-of-ukraines-war/ar-AAJgNsz?ocid=chopendata']

In [37]:
train_small_htmls = []

for url in train_small_urls:
    train_small_htmls.append('https://assets.msn.com/labs/mind/' + url.split('/')[-1].split('?')[0][-7:] + '.html')

print(len(train_small_htmls))
train_small_htmls[:3]

51282


['https://assets.msn.com/labs/mind/AAGH0ET.html',
 'https://assets.msn.com/labs/mind/AAB19MK.html',
 'https://assets.msn.com/labs/mind/AAJgNsz.html']

In [7]:
train = pd.read_csv('/Users/baeyuna/Documents/SNU_Dlab/Data/MINDlarge_train/news.tsv', delimiter='\t', header=None)
print(train.shape)
train.head()

(101527, 8)


,0,1,2,3,4,5,6,7
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."


In [10]:
train.iloc[:5,5].tolist()

['https://assets.msn.com/labs/mind/AAGH0ET.html',
 'https://assets.msn.com/labs/mind/AABmf2I.html',
 'https://assets.msn.com/labs/mind/AAB19MK.html',
 'https://assets.msn.com/labs/mind/AAISxPN.html',
 'https://assets.msn.com/labs/mind/AAJgNsz.html']

In [ ]:
train_urls = train.loc[:,5]

In [11]:
dev = pd.read_csv('/Users/baeyuna/Documents/SNU_Dlab/Data/MINDlarge_dev/news.tsv', delimiter='\t', header=None)
print(dev.shape)
dev.head()

(72023, 8)


,0,1,2,3,4,5,6,7
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
3,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
4,N75236,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."


In [ ]:
def getReviews(idx):
    reviews_list = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
    }
    
    for page in [0,1,2]:
        try: # 페이지가 있으면
            url = f'https://store.naver.com/restaurants/detail?entry=plt&id={idx}&tab=receiptReview&tabPage={page}'
            response = requests.get(url, headers = headers)
            root = lxml.html.fromstring(response.text)
            
            nicks = [elem.text_content().strip() for elem in root.cssselect('div.reviewer > a')]
            reviews = [elem.text_content().strip() for elem in root.cssselect('div.review_txt')]
            stars = [elem.attrib['style'] for elem in root.cssselect('span.bg > span.value')]
            herfs = [elem.attrib['href'].split('my/')[1].split('/')[0] for elem in root.cssselect('a.item')]
            
            value = list(map(list, zip(nicks, reviews, stars, herfs)))
            [x.append(idx) for x in value]
            reviews_list.extend(value)
        except: # 페이지가 없으면
            continue

    return reviews_list

In [12]:
import os

In [16]:
train.head()

,0,1,2,3,4,5,6,7
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."


In [19]:
train.iloc[0,6]

'[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]'

In [17]:
start_urls = []
with open('/Users/baeyuna/Documents/SNU_Dlab/Data/MINDlarge_train/news.tsv', 'r') as f:
    for l in f:
        _, _, _, _, _, url, _, _ = l.strip('\n').split('\t')
        start_urls.append(url)